In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from glob import glob
import pickle

import numpy as np
import pandas as pd

#TEXT PROCESSING
import nltk
import re
import codecs
import unidecode
#pip install unidecode
import mpld3
# pip install mpld3
import stop_words
# pip install stop-words
from nltk import SnowballStemmer, pos_tag, word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

#SKLEARN
from sklearn.feature_extraction.text import *
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans


** Lecture des données **

In [2]:
#download cv in a list
def load_cv_list(nombre):
    path = '/Users/mehdiregina/FilRouge/Maha/data/Txt/'
    liste_paths = [path+directory for directory in os.listdir(path)]
    liste_cv = []
    liste_files = []
    for path in liste_paths:
        filenames = sorted(glob(os.path.join(path,"*.txt")))
        for file in filenames[:nombre]:
            liste_cv.append(open(file).read())
            liste_files.append(file)
    return liste_cv,liste_files,liste_paths

In [3]:
liste_cv,liste_files,liste_paths = load_cv_list(200)
liste_cv[0]

"INFORMATICIEN DÉVELOPPEMENT ET\nRÉSEAUX\n\nDéveloppeur Intégrateur Web\n\nÉragny (95) - Email me on Indeed: indeed.com/r/d7e8913ed00d0384\n\nAujourd’hui, je suis en recherche d'une opportunité sur un poste de développeur ou d’intégrateur web afin de\nmonter toujours plus en compétence et d’approfondir les bases solide que j’ai acquis en formation.\n\nEXPÉRIENCE\n\nINFORMATICIEN DÉVELOPPEMENT ET RÉSEAUX\n\nLeGrandCercle95  -  Éragny (95) -\n\nnovembre 2016 - juin 2017\n\nInformaticien de l'entreprise, mes missions était de gérer les différent problème dans l'entreprise. Mise en\nplace d'un antivirus serveur, sauvegarde Nas... Créé et gérer les droits sur l'Active Directory. Paramétrer des\nclients léger ainsi que du matériel informatique comme des imprimantes ou des étiqueteuse en IP fixe.\n\nRéajustement du code html et css sur le site grand public selon les normes w3c.\nCréation d'une source ODBC\nCréation d'un code en php - sql afin de récupéré des données librairie sur un site four

In [4]:
print(len(liste_cv), "cvs")

2200 cvs


#### Suppression des sauts de ligne et text.lower()

In [5]:
import string,re
regex = re.compile('[%s]' % '(\\n)*(\\x0c)*')
def del_line_feed(s):  
    """Delete \n in the text"""
    return regex.sub(' ', s)

In [6]:
liste_cv = [del_line_feed(text).lower() for text in liste_cv]
liste_cv[0]

"informaticien développement et réseaux  développeur intégrateur web  éragny  95  - email me on indeed: indeed.com/r/d7e8913ed00d0384  aujourd’hui, je suis en recherche d'une opportunité sur un poste de développeur ou d’intégrateur web afin de monter toujours plus en compétence et d’approfondir les bases solide que j’ai acquis en formation.  expérience  informaticien développement et réseaux  legrandcercle95  -  éragny  95  -  novembre 2016 - juin 2017  informaticien de l'entreprise, mes missions était de gérer les différent problème dans l'entreprise. mise en place d'un antivirus serveur, sauvegarde nas... créé et gérer les droits sur l'active directory. paramétrer des clients léger ainsi que du matériel informatique comme des imprimantes ou des étiqueteuse en ip fixe.  réajustement du code html et css sur le site grand public selon les normes w3c. création d'une source odbc création d'un code en php - sql afin de récupéré des données librairie sur un site fournisseur pour les enregis

#### Suppression ponctuation

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
#le maintient de la ponctuation pertube le stop words, apostrophe gérée dans text_treatment
regex = re.compile('[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_{|}~')) 

def del_punct(s):  
    """Delete punctuation in the text"""
    return regex.sub('', s)

In [9]:
#test 
liste_cv_no_punc = [del_punct(text) for text in liste_cv]
liste_cv_no_punc[0]

"informaticien développement et réseaux  développeur intégrateur web  éragny  95   email me on indeed indeedcomrd7e8913ed00d0384  aujourd’hui je suis en recherche d'une opportunité sur un poste de développeur ou d’intégrateur web afin de monter toujours plus en compétence et d’approfondir les bases solide que j’ai acquis en formation  expérience  informaticien développement et réseaux  legrandcercle95    éragny  95    novembre 2016  juin 2017  informaticien de l'entreprise mes missions était de gérer les différent problème dans l'entreprise mise en place d'un antivirus serveur sauvegarde nas créé et gérer les droits sur l'active directory paramétrer des clients léger ainsi que du matériel informatique comme des imprimantes ou des étiqueteuse en ip fixe  réajustement du code html et css sur le site grand public selon les normes w3c création d'une source odbc création d'un code en php  sql afin de récupéré des données librairie sur un site fournisseur pour les enregistré en fiche xml cré

#### Reconnaissance langage

In [10]:
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens] #from text get list of word in minuscule

    
    for language in stopwords.fileids(): # pour chaque langue
        stopwords_set = set(stopwords.words(language)) #je mets les stop words du langage dans un set
        words_set = set(words) #je mets les mots de mon texte dans un set
        #je prends l'intersection entre les mots de mon texte et les mots du stopwords dans le langage donné
        common_elements = words_set & stopwords_set
        
        #je compute mon score comme le nombre d'éléments en communs dictionnaire [langage : score]
        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios

In [11]:
import nltk
nltk.download('stopwords')
stopwords.fileids()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mehdiregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish',
 'turkish']

In [12]:
def get_cv_langue(liste_cv, liste_filenames,language) :
    """Return resume witten in the specified language in parameter"""
    liste_2 = []
    listeFileNames = []
    i=0
    for cv in liste_cv :
        if max(_calculate_languages_ratios(cv),key =_calculate_languages_ratios(cv).get)=='french':
            liste_2.append(cv)
            listeFileNames.append(liste_filenames[i])
        i+=1
    return liste_2,listeFileNames

In [13]:
liste_cv_fr, liste_fileNames_fr = get_cv_langue(liste_cv_no_punc,liste_files,'french')

nb_cv = len(liste_cv_no_punc)
nb_cv_fr = len(liste_cv_fr)

print("proportion cv french :",1- ((nb_cv-nb_cv_fr)/nb_cv))


proportion cv french : 0.730909090909091


In [14]:
nb_cv_fr = len(liste_cv_fr)

print('nombre de CV français:', len(liste_fileNames_fr))


nombre de CV français: 1608


** text treatment **

In [15]:
def text_treatment (text):
    text = text.lower()
    text = text.replace("\x00", '').replace("\x01", '').replace("\x02", '').replace("\x03", '') \
    .replace("\x04", '').replace("\x05", '').replace("\x06", '').replace("\x07", '').replace("\x08", '') \
    .replace("\x0e", '').replace("\x11", '').replace("\x12", '').replace("\x10", '').replace("\x19", '') \
    .replace("\x1b", '').replace("\x14", '').replace("\x15", '').replace('/', ' ').replace('=', ' ').replace("〓", "") \
    .replace("»", "").replace("«", "").replace("¬", "").replace('`', '').replace (" -", " ").replace("•", "")\
    .replace("l'", "").replace("l’", "").replace("l´", "").replace("d’", "").replace("d'", "").replace("d´","")\
    .replace("j’", "").replace("j'", "").replace("j´","").replace("n’", "").replace("n'", "").replace("n´","")\
    .replace("”", "").replace("~", "").replace("§", "").replace("¨", "").replace("©", "").replace("›", "")\
    .replace("₋", "").replace("→", "").replace("⇨", "").replace("∎", "").replace("√", "").replace("□", "")\
    .replace("*", "").replace("&", "").replace("►", "").replace("◊", "").replace("☞", "").replace("#", "")\
    .replace("%", "").replace("❖", "").replace("➠", "").replace("➢", "").replace("", "").replace("✓", "") \
    .replace("√", "").replace("✔", "").replace("♦", "").replace("◦", "").replace("●", "").replace("▫", "")\
    .replace("▪", "").replace("…", "").replace("þ", "").replace("®", "").replace('', '').replace("...", "")
    text = unidecode.unidecode(text) # remove accent
    return text

In [16]:
#On supprime les caractères étranges et accents
liste_cv_treated = [text_treatment(text) for text in liste_cv_fr]
liste_cv_treated[0]

"informaticien developpement et reseaux  developpeur integrateur web  eragny  95   email me on indeed indeedcomrd7e8913ed00d0384  aujourhui je suis en recherche une opportunite sur un poste de developpeur ou integrateur web afin de monter toujours plus en competence et approfondir les bases solide que ai acquis en formation  experience  informaticien developpement et reseaux  legrandcercle95    eragny  95    novembre 2016  juin 2017  informaticien de entreprise mes missions etait de gerer les different probleme dans entreprise mise en place un antivirus serveur sauvegarde nas cree et gerer les droits sur active directory parametrer des clients leger ainsi que du materiel informatique comme des imprimantes ou des etiqueteuse en ip fixe  reajustement du code html et css sur le site grand public selon les normes w3c creation une source odbc creation un code en php  sql afin de recupere des donnees librairie sur un site fournisseur pour les enregistre en fiche xml creation de bannieres pou

#### Gestion des stop words

In [17]:
#generate stopwords
stop_words_py = set(stop_words.get_stop_words('french'))

# attention certains stop words pourraient être utiles par la suite
stopwords_set_manuel = set(["an", "ans", 'les', 'moins', 'd\'un','janvier', 'fevrier', 'février', 'mars', 'avril', \
                 'mai', 'juin', 'juillet', 'aout', 'août', 'septembre', 'octobre', 'novembre', 'décembre', \
                  'decembre', 'moins', 'mise', 'universit\xc3\xa9', 'université', 'universite', 'ion','sage', \
                  'o', 'rac', 'vers', 'via', 'p\xc3\xa9rim\xc3\xa8tre', 'périmètre','et','paris','x',"\x00",\
                          "\x01","\x02", "\x03","\x04","\x05","\x06","\x07","\x08","\x09","\x0e","\x0e","\x11",\
                           "\x12","\x13","\x14","\x15","\x16","\x17","\x18","\x19","transport","puis","lieu",\
                           "adresse","entre",'dun','dune','chez','boulognebillancourt','bt','etc','recrutement','main',\
                           'and', 'paie','paiement','environ','place','france','paris','mois','mobile','mobiles',\
                           'nanterre','source','sources','concerne','concernant','of','non','notes','rh','minimum',\
                           'maximum','bac','site','sites','actuellement','telephone','telephonique','telephoniques','ca','demenager',\
                           'demenagement','participer','participation','lycee','baccalaureat','lien','liens','in',\
                           'indeed','email','indeedcomrd7e8913ed00d0384','aujourhui','afin','toujours','enterprise',\
                           "guide","10g","11g","9i",'ad','v10','v2','v3','v5','v6','v8','v9',])
stop_words_main = stop_words_py | stopwords_set_manuel
stop_words_main = list(stop_words_main)
print("taille stop words liste : ", len(stop_words_main))

taille stop words liste :  368


In [18]:
stop_words_main

['leur',
 'v5',
 'lien',
 'fussions',
 'eussiez',
 'telephone',
 'eussions',
 'fut',
 'site',
 'toujours',
 'mois',
 'étaient',
 'était',
 'on',
 'fûmes',
 'mais',
 'quand',
 '9i',
 'même',
 'hors',
 'février',
 'votre',
 'pour',
 'adresse',
 'été',
 'v8',
 'eûtes',
 'si',
 'seras',
 'es',
 'aie',
 'sans',
 'peu',
 'v10',
 'v6',
 'eût',
 'baccalaureat',
 'droite',
 'france',
 'dois',
 '\x17',
 '\x18',
 'vous',
 'serait',
 'avril',
 'aient',
 'eûmes',
 'pas',
 'universitÃ©',
 'seraient',
 'que',
 'valeur',
 'le',
 'quel',
 'depuis',
 'enterprise',
 'pourquoi',
 'devront',
 'd',
 'vos',
 'ca',
 'tous',
 'transport',
 'aurais',
 'afin',
 'peut',
 'personne',
 'les',
 '\x01',
 'ayez',
 'a',
 'rac',
 'and',
 'fais',
 'ta',
 'mobile',
 'chaque',
 '\x16',
 'juillet',
 'par',
 'personnes',
 'étions',
 'v9',
 'actuellement',
 'sources',
 'aies',
 'y',
 'juin',
 'main',
 'nous',
 'cette',
 'eux',
 'sage',
 'of',
 'auriez',
 'sien',
 'fussiez',
 '\x12',
 'plupart',
 'de',
 'ans',
 'êtes',
 'dos',

In [19]:
#voir si utile
def remove_stopwords(text,stopwords_list):
    text_temp = " ".join(text.split())+" "
    for word in stopwords_list:
        text_temp = text_temp.replace(" "+word+" ", " ")
    return text_temp

In [20]:
liste_cv_no_stop = [remove_stopwords(text,stop_words_main) for text in liste_cv_treated]
liste_cv_no_stop[0]

"informaticien developpement reseaux developpeur integrateur web eragny 95 recherche opportunite poste developpeur integrateur web monter plus competence approfondir bases solide acquis formation experience informaticien developpement reseaux legrandcercle95 eragny 95 2016 2017 informaticien entreprise missions etait gerer different probleme entreprise antivirus serveur sauvegarde nas cree gerer droits active directory parametrer clients leger ainsi materiel informatique imprimantes etiqueteuse ip fixe reajustement code html css grand public selon normes w3c creation odbc creation code php sql recupere donnees librairie fournisseur enregistre fiche xml creation bannieres differents evenements photoshop formation logiciel photoshop charge clientele europcar commercial saintouenaumone 95 2008 2016 missions principales qualite service assurer accueil clients respect charte agence gestion clients traiter ensemble appels analyser besoins client assurer suivi clientele logistique administrat

In [21]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french")

In [22]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #print(tokens)
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [23]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for text in liste_cv_no_stop:
    allwords_stemmed = tokenize_and_stem(text) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(text)
    totalvocab_tokenized.extend(allwords_tokenized)

In [24]:
liste_cv_cleaned = [tokenize_only(text) for text in liste_cv_no_stop]
liste_cv_no_num = [' '.join(cv) for cv in liste_cv_cleaned]

In [25]:
#on constate que les caracteres numériques sont biens supprimés
liste_cv_no_num[0] 

"informaticien developpement reseaux developpeur integrateur web eragny recherche opportunite poste developpeur integrateur web monter plus competence approfondir bases solide acquis formation experience informaticien developpement reseaux legrandcercle95 eragny informaticien entreprise missions etait gerer different probleme entreprise antivirus serveur sauvegarde nas cree gerer droits active directory parametrer clients leger ainsi materiel informatique imprimantes etiqueteuse ip fixe reajustement code html css grand public selon normes w3c creation odbc creation code php sql recupere donnees librairie fournisseur enregistre fiche xml creation bannieres differents evenements photoshop formation logiciel photoshop charge clientele europcar commercial saintouenaumone missions principales qualite service assurer accueil clients respect charte agence gestion clients traiter ensemble appels analyser besoins client assurer suivi clientele logistique administratif s'assurer disponibilite ve

In [26]:
len(liste_cv_no_num)

1608

In [27]:
def del_word_freq(max_idf,min_idf,list_texts):
    
    tf_vect = TfidfVectorizer(stop_words=stop_words_main,max_df=max_idf,min_df=min_idf,\
                           preprocessor=text_treatment,tokenizer=tokenize_only)
    bow_idf_indeed = tf_vect.fit_transform(list_texts)
    vocab_liste_idf = tf_vect.get_feature_names()
    
    liste_texte_new =[]
    prop = 0
    for text in list_texts:
        liste_mot = [mot for mot in text.split(' ') if mot in vocab_liste_idf]
        liste_texte_new.append(' '.join(liste_mot))
        prop += len(liste_mot)/len(text.split(' '))
        
    return liste_texte_new, (prop/len(list_texts))

In [28]:
liste_texte_new, prop = del_word_freq(0.80,0.05,liste_cv_no_num)

In [29]:
prop

0.5074057269038234

In [30]:
liste_texte_new[0]

'informaticien developpement reseaux developpeur web recherche poste developpeur web plus competence bases formation informaticien developpement reseaux informaticien entreprise missions gerer entreprise serveur sauvegarde gerer active directory clients ainsi materiel informatique ip html css normes creation creation php sql donnees xml creation differents evenements photoshop formation logiciel photoshop charge clientele commercial missions qualite service assurer accueil clients respect agence gestion clients ensemble appels analyser besoins client assurer suivi clientele logistique gerer agence gestion caisse formation developpeur web centre formation creation differents presentation ecommerce developpeur web stage developpement commercial stagiaire assurance stagiaire formation niveau developpeur web'

In [31]:
liste_cv_no_num[0]

"informaticien developpement reseaux developpeur integrateur web eragny recherche opportunite poste developpeur integrateur web monter plus competence approfondir bases solide acquis formation experience informaticien developpement reseaux legrandcercle95 eragny informaticien entreprise missions etait gerer different probleme entreprise antivirus serveur sauvegarde nas cree gerer droits active directory parametrer clients leger ainsi materiel informatique imprimantes etiqueteuse ip fixe reajustement code html css grand public selon normes w3c creation odbc creation code php sql recupere donnees librairie fournisseur enregistre fiche xml creation bannieres differents evenements photoshop formation logiciel photoshop charge clientele europcar commercial saintouenaumone missions principales qualite service assurer accueil clients respect charte agence gestion clients traiter ensemble appels analyser besoins client assurer suivi clientele logistique administratif s'assurer disponibilite ve

### Doc2Vec essaie 1

Pourquoi ne fonctionnerait-il pas ?

-> Le contexte n'explique pas suffisemment le mot à prédire, en quelques mots on passe "du coq à l'âne" sur un CV. Or le word2vec émet l'hypothèse que le contexte explique le mot.

Est ce qu'un prétraitement est nécessaire ? OUI il est conseillé => Tokenizing et eventuellement Stemming, stop word removing (il n'apportent rien au contexte), punctuation removing, change numeric into letters.

Comment l'améliorer ? -> Faire le Word2vec sur une sous partie du CV
-> Retirer la partie Centre d'intérets (ajoute du bruit dans la compréhension du contexte du cv)
-> Certaines parties correspondent à un listing de mots Secteurs d'activité, Outils : intérêt de conserver ? -> Peut-il trouver un contexte pour le mots listés

Attention le dernier envoie de CV de Talan (CV_3) ne contient pas de phrases, uniquement des listing

In [35]:
import gensim
import logging
import re
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

class Documents(object):
    def __init__(self, documents, labels):
        self.documents = documents
        self.labels = labels

    def __iter__(self):
        for i, doc in enumerate(self.documents):
            yield TaggedDocument(words = doc, tags = [self.labels[i]])

def create_documents_doc2vec(liste_cv,liste_labels):
    
    cvdocs = liste_cv
    count = len(cvdocs)
    preprocessedCv = []
    preprocessedLabels = []
    duplicate_dict = {}

    ilabel = 0
    for t in cvdocs:
        if t not in duplicate_dict:
            duplicate_dict[t] = True
            preprocessedCv.append(t)
            preprocessedLabels.append(liste_labels[ilabel].split('/')[-1].split('.')[0]) 
        ilabel +=1

    documents = Documents(preprocessedCv,preprocessedLabels)
    return documents

documents = create_documents_doc2vec(liste_texte_new,liste_fileNames_fr) 
len(documents.labels)

1392

Ajouter un tf-idf pour trier les mots au préalable, étape de preprocessing supplémentaire

In [36]:
def train_doc2vec(documents,model_name):
    #size : dim du vector représentant les mots : représente la dim de l'hidden layer
    #window : max distance between current word and the word 2 predict <=> taille du contexte ?
    model = Doc2Vec(size=50, dbow_words= 1, dm=0, iter=1,  window=3, seed=1337, min_count=0, workers=4,alpha=0.025, 
                    min_alpha=0.025)

    model.build_vocab(documents)
    #training of model
    #how to get model error on the train ?
    for epoch in range(100):
        print('iteration '+str(epoch+1))
        model.train(documents,total_examples=model.corpus_count,epochs=model.iter)
        model.save(model_name)
        model.alpha -= 0.002
        model.min_alpha = model.alpha
    return model


In [38]:
model = train_doc2vec(documents,'Doc2vec300.model')
#docvec = model.docvecs['./data/Txt/business_analyst/cv_business analyst_541.txt'] #if string tag used in training
#len(model.docvecs)

iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [39]:
documents.labels[-400:]

['cv_chef_de_projet_informatique_186',
 'cv_chef_de_projet_informatique_187',
 'cv_chef_de_projet_informatique_188',
 'cv_chef_de_projet_informatique_189',
 'cv_chef_de_projet_informatique_19',
 'cv_chef_de_projet_informatique_190',
 'cv_chef_de_projet_informatique_191',
 'cv_chef_de_projet_informatique_192',
 'cv_chef_de_projet_informatique_193',
 'cv_chef_de_projet_informatique_194',
 'cv_chef_de_projet_informatique_195',
 'cv_chef_de_projet_informatique_196',
 'cv_chef_de_projet_informatique_197',
 'cv_chef_de_projet_informatique_198',
 'cv_chef_de_projet_informatique_199',
 'cv_chef_de_projet_informatique_2',
 'cv_chef_de_projet_informatique_20',
 'cv_chef_de_projet_informatique_200',
 'cv_chef_de_projet_informatique_201',
 'cv_chef_de_projet_informatique_202',
 'cv_chef_de_projet_informatique_203',
 'cv_chef_de_projet_informatique_204',
 'cv_chef_de_projet_informatique_205',
 'cv_chef_de_projet_informatique_206',
 'cv_chef_de_projet_informatique_207',
 'cv_chef_de_projet_informati

In [40]:
model.docvecs.most_similar('cv_data scientist_26')

[('cv_data analyst_116', 0.9189307689666748),
 ('cv_business analyst_181', 0.9111403822898865),
 ('cv_business intelligence_19', 0.9026414155960083),
 ('cv_DBA_145', 0.8998908996582031),
 ('cv_data analyst_105', 0.8968216776847839),
 ('cv_business intelligence_24', 0.8936462998390198),
 ('cv_data analyst_117', 0.8924216032028198),
 ('cv_informaticien_230', 0.8885915279388428),
 ('cv_data analyst_109', 0.8881703615188599),
 ('cv_data scientist_135', 0.8872749209403992)]

In [41]:
#start testing
#printing the vector of document at index 1 in docLabels
docvec = model.docvecs[1]
#print(docvec)
#printing the vector of the file using its name
#docvec = model.docvecs['1.txt'] #if string tag used in training
#print(docvec)
#to get most similar document with similarity scores using document-index
similar_doc = model.docvecs.most_similar(14) 
#print(similar_doc)
#to get most similar document with similarity scores using document- name
#sims = model.docvecs.most_similar('1.txt')
#print(sims)
#to get vector of document that are not present in corpus 
#docvec = d2v_model.docvecs.infer_vector(‘war.txt’)
#print docvec


In [42]:
import nltk, math, codecs
from gensim.models import Doc2Vec
from nltk.cluster.kmeans import KMeansClusterer

NUM_CLUSTERS = 5
def kmeans_doc2Vec(model_name):
    
    model = Doc2Vec.load(model_name)

    kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25)
    assigned_clusters = kclusterer.cluster(model.docvecs, assign_clusters=True)
    return assigned_clusters


In [43]:

def preprocess_document(text):
    #text = preprocess(text)
    return ''.join([x if x.isalnum() or x.isspace() else " " for x in text ]).split()

def get_titles_by_cluster(id,clusters):
    list = []
    for x in range(0, len(assigned_clusters)):
        if (clusters[x] == id):
            list.append(preprocessedCv[x])
    return list

def get_topics(titles):
    from collections import Counter
    words = [preprocess_document(x) for x in titles]
    words = [word for sublist in words for word in sublist]
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    count = Counter(filtered_words)
    print(count.most_common()[:20])


def cluster_to_topics(id,clusters):
    get_topics(get_titles_by_cluster(id,clusters))


In [44]:
  assigned_clusters = kmeans_doc2Vec("Doc2vec300.model")
for cluster in range(NUM_CLUSTERS):
    print(cluster_to_topics(cluster,assigned_clusters))

KeyError: "tag '1392' not seen in training corpus/invalid"

In [ ]:
 model.docvecs.index_to_doctag(15)

#### TEST CV TALAN

In [49]:
def load_cv_list(nombre):
    path = './data/Txt/'
    liste_paths = [path+directory for directory in os.listdir(path)]
    liste_cv = []
    liste_files = []
    for path in liste_paths:
        filenames = sorted(glob(os.path.join(path,"*.txt")))
        
        for file in filenames[:nombre]:
            liste_cv.append(open(file).read())
            liste_files.append(file)
    return liste_cv,liste_files,liste_paths

In [50]:
liste_cv_talan = []
path = "../Mehdi/data_talan"
filenames = sorted(glob(os.path.join(path,"*.txt")))
for file in filenames:
    liste_cv_talan.append(open(file).read())

In [84]:
len(filenames)

166

In [45]:
#On supprime les caractères étranges et accents
texts_treated_talan = [text_treatment(remove_stopwords(text,stop_words_main)) for text in liste_cv_talan]

In [46]:
liste_cv_docvec = [tokenize_only(text) for text in texts_treated_talan ]
liste_cv_docvec_noNum = [' '.join(cv) for cv in liste_cv_docvec]


In [66]:
liste_cv_docvec_noNum[15]

'aau consultant decisionnel junior sein cabinet talan solution son collectif permettra rejoindre facilement equipe analyse decisionnelle aau consultant junior data visualization tableau software competences outils methodes travail mode agile suivant methodologie scrum aide outil jira communication ecoute entraide travail equipe outils tableau software sql pl-sql methodes agiles scrum ||kanban r academique expertise technique analyse donnees rendu visuel clair creation modification base donnees creation diagrammes uml langages programmation sql t-sql triggers procedures stockees mdx academique html css javascript xml secteurs activite metier telecommunication industrie chimique formations certifications diplome ingenieur genie informatique option business intelligence e.i.s.t.i ecole internationale sciences traitement information cergy experiences ineos france juin projet reecriture logiciel calculs cokage vapocraqueur role mission programmation informatique contributions modification g

In [86]:
def subpart_cv(liste_cv,liste_filenames,part=0):
    """part can be :
            0 - competence
            1 - experience"""
    assert part==0 or part==1
    liste_sub = []
    for i,text in enumerate(liste_cv) :
        #si on a le mot experience au début
        if "experience professionnelle" in text :
            liste_sub.append(text.split('experience professionnelle')[part])
        elif "experiences" in text :
            liste_sub.append(text.split('experiences')[part])
        else :
            liste_filenames.pop(i)
    
    return liste_sub, liste_filenames
        

In [87]:
liste_competences, filenames = subpart_cv(liste_cv_docvec_noNum,filenames)

In [89]:
len(liste_competences)

163

On en perd 3 => 3 CVs ne contiennent ni "experiences" ni experience professionnelle, à traiter...

In [95]:
documents_talan = create_documents_doc2vec(liste_competences,filenames)
len(documents_talan.documents)

158

On en perd 5 => 5 CVs dupliqués

In [96]:
documents_talan.labels

['ADECV TALAN',
 'AGNCV TALAN',
 'AKACV TALAN',
 'APHCV TALAN',
 'APICV TALAN',
 'BGHCV TALAN',
 'BKICV TALAN',
 'BLACV TALAN',
 'BLPCV TALAN',
 'BPUCV TALAN',
 'CAPCV TALAN',
 'CLSCV TALAN',
 'CV - TalanSolutions - AGU',
 'CV - TalanSolutions - CJD',
 'CV - TalanSolutions - DPE',
 'CV AAU_TS2017',
 'CV HME_TS2017',
 'CV LKH - TALAN-2017',
 'CV NPO',
 'CV TalanSolutions ABAH',
 'CV TalanSolutions ACH',
 'CV TalanSolutions AIQ',
 'CV TalanSolutions AMO',
 'CV TalanSolutions ANDO',
 'CV TalanSolutions ATR',
 'CV TalanSolutions BILI',
 'CV TalanSolutions CPOU',
 'CV TalanSolutions DJE',
 'CV TalanSolutions DJS',
 'CV TalanSolutions FGA',
 'CV TalanSolutions ICHO',
 'CV TalanSolutions JBOU',
 'CV TalanSolutions LVE',
 'CV TalanSolutions MCHI',
 'CV TalanSolutions MSME',
 'CV TalanSolutions NAB',
 'CV TalanSolutions NPRI',
 'CV TalanSolutions RGO',
 'CV TalanSolutions RGUI',
 'CV TalanSolutions RVA',
 'CV TalanSolutions SEDA',
 'CV TalanSolutions XSUN',
 'CV TalanSolutions YGO',
 'CV _Talan

In [97]:
model_talan = train_doc2vec(documents_talan,'Doc2vec300_talan.model')
model_talan

/Users/mehdiregina/anaconda/lib/python3.6/site-packages/gensim/models/doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/Users/mehdiregina/anaconda/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/Users/mehdiregina/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if sys.path[0] == '':


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [93]:
sims = model_talan.docvecs.most_similar("CV_DS_1")

print(sims)

[('CV_YWA_TS2017', 0.9890708923339844), ('CV_DS_2', 0.986083984375), ('CAPCV TALAN', 0.9843279719352722), ('CLSCV TALAN', 0.9839305877685547), ('FGICV TALAN', 0.9823065996170044), ('CV_MDM_10', 0.9819942712783813), ('Talan - CV AGIR', 0.9805023074150085), ('CV_TalanSolutions_FVAL', 0.980362057685852), ('Talan - CV EGTA', 0.980359673500061), ('CV HME_TS2017', 0.978887140750885)]


In [ ]:
assigned_clusters = kmeans_doc2Vec('Doc2vec300_talan.model')
for cluster in range(NUM_CLUSTERS):
    print(cluster_to_topics(cluster,assigned_clusters))

non concluant